In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import accuracy_score

___
# Ames Housing Dataset: Model **TRAINING**
___

In [53]:
#store encoded dataset in variable for readability
encode_path = ("/Users/sa14/Desktop/HousePricesKaggle/data/training_encoded_dataset.csv")

#load as a dataframe
train_encode = pd.read_csv(encode_path)

#verify encoding
train_encode.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NA,BsmtQual_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NA,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NA,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NA,BsmtFinType2_Rec,BsmtFinType2_Unf,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NA,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,706,150,856,856,854,1710,1,2,1,3,8,0,2003.0,2,548,0,61,2,2008,208500,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,6,8,1976,1976,978,284,1262,1262,0,1262,0,2,0,3,6,1,1976.0,2,460,298,0,5,2007,181500,0.0,0.0,0.0,1

#### The function below will determine Root Mean Squared Error (rmse) of the Logistic Regression, GradientBoostingRegressor, and RandomForestRegressor algorithms.

#### A **lower** RMSE indicates better model performance.

#### We'll use a RobustScaler instead of a StandardScaler to preserve the shape of the distribution and scale the numeric features while being robust to outliers

#### ***"Robust to outliers"** means the results of the method are resistant to the undue influence of extreme outliers*

In [54]:
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.model_selection import train_test_split
# Create a list of Features to be stored in "X"

# The Features list will not include "SalePrice" as that will be the target feature
X = train_encode.drop(columns=["SalePrice"])  

# Target: "SalePrice" column
y = train_encode["SalePrice"] 

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, 
                                                    random_state=42)

# Set up Robust Scaler then scale existing data
from sklearn.preprocessing import RobustScaler

scaler= RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define function to train, evaluate, and print results
def eval_train_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r_mean_square_err = rmse(y_test, y_pred)
    
    print(f"\n{model_name} Root Mean Squared Error:", r_mean_square_err)
    return r_mean_square_err

In [55]:
#Get count of rows & columns for X_test and X_train
print ("Rows and Columns of X_test:\n", X_test.shape, "\n\nRows and Columns of the X_train:\n", X_train.shape) 

Rows and Columns of X_test:
 (482, 190) 

Rows and Columns of the X_train:
 (978, 190)


#### The data will be regularized to prevent overfitting using both L1 (Lasso) and L2 (Ridge) regularization. L1 adds a penalty based on the sum of absolute values of the coefficients. L2 adds a penalty based on the sum of squares of the coefficients. 

#### Through using both L1 and L2 penalities, we allow the models to do feature selection and coefficient shrinking.

####  The solver parameter displays the specific optimization algorithm used to find the model's coefficients. *Liblinear* was chosen because it works well with both L1 & L2 regularization. *Saga* supports L1 regularization, is good for feature selection, large datasets and multiclass concerns.

In [56]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error as rmse
from scipy.stats import uniform, randint

# Store RMSE output from eval_train_model function
training_model_results = {}

#Set general parameters for the 4 models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost Regressor": xgb.XGBRegressor(random_state=42)
}

for name, model in models.items():
    r_mean_square_err =  eval_train_model(model, name)
    training_model_results[name] = {"Root Mean Squared Error": r_mean_square_err}

#hyperparameter grids for the 4 models
param_grids = {
    "Logistic Regression": {
        'C': uniform(0.1, 9.9),  # 0.1 to 10
        'solver': ['liblinear', 'saga'],
        'max_iter': [300, 600]
    },
    "Random Forest": {
        'n_estimators': randint(50, 200),
        'max_depth': randint(3, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 10),
        'bootstrap': [True, False]
    },
    "Gradient Boosting": {
        'n_estimators': randint(50, 200),
        'learning_rate': uniform(0.01, 0.19),
        'max_depth': randint(3, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 10)
    },
    "XGBoost Regressor": {
        'objective': ['reg:squarederror'],
        'n_estimators': randint(100, 1000),
        'learning_rate': uniform(0.01, 0.19),
        'max_depth': randint(3, 15),
        'min_child_weight': randint(1, 10),
        'gamma': uniform(0, 0.5),
        'subsample': uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.3, 0.7),
        'reg_alpha': uniform(0, 0.1),
        'reg_lambda': uniform(1, 9),
        'n_jobs': [-1],
        'random_state': [42]
    }
}

# Determine best model through setting "min" = lowest aka best performing RSME
top_model_name = min(training_model_results, key=lambda k: training_model_results[k]["Root Mean Squared Error"])
top_model = models[top_model_name]
param_distributions = param_grids[top_model_name]


print(f"\nBest Performing Home SalePrice Prediction Model: {top_model_name}")


Logistic Regression Root Mean Squared Error: 50198.73415311076

Random Forest Root Mean Squared Error: 31115.980610420287

Gradient Boosting Root Mean Squared Error: 29826.085065437717

XGBoost Regressor Root Mean Squared Error: 29198.29296875

Best Performing Home SalePrice Prediction Model: XGBoost Regressor


___
## **Hyperparameter Tuning of the XGBoost Regressor Model with Randomized Search** 
___

The scoring in RandomSearch is set to negative mean sqaured error for minimization.

(**Minimization** is a method of randomization that allocates subjects to the training group that best maintains balance in predictive features)

In [57]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error as rmse
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV

# Perform randomized search
random_search = RandomizedSearchCV(
    estimator=top_model,
    param_distributions=param_distributions,
    n_iter=100,
    scoring='neg_mean_squared_error',  #scoring is set to negative mean squared error for minimization
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

#Display optimal hyperparameters for the XGBoost Regressor Model with RandomizedSearch
print(f"\nBest Hyperparameters for {top_model}: {random_search.best_params_}\n")

# Retrain with optimal hyperparameters on encoded TRAINING housing dataset
encode_path = ("/Users/sa14/Desktop/HousePricesKaggle/data/training_encoded_dataset.csv")
train_encode = pd.read_csv(encode_path)

# The Features list will not include "SalePrice" as that will be the target feature
X_op = train_encode.drop(columns=["SalePrice"])  

# Target: "SalePrice" column
y_op = train_encode["SalePrice"]

X_op_train, X_op_test, y_op_train, y_op_test = train_test_split(train_encode, y_op, 
                                                                test_size=0.33, random_state=42)

# ROBUST Scale encoded TRAINING dataset
from sklearn.preprocessing import RobustScaler
X_op_train = scaler.fit_transform(X_op_train)
X_op_test = scaler.transform(X_op_test)

# Final training with hyperparameter tuned encoded training set
final_trained_model = random_search.best_estimator_
final_trained_model.fit(X_op_train, y_op_train)

# Evaluate final model's root mean squared error
true_rmse = eval_train_model(final_trained_model, f"{top_model} Tuned with RandomizedSearch")

print("\nFinal Model Results:\n")
print(f"Root Mean Squared Error: {true_rmse:.4f}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END colsample_bytree=0.5621780831931538, gamma=0.4753571532049581, learning_rate=0.14907884894416698, max_depth=7, min_child_weight=7, n_estimators=221, n_jobs=-1, objective=reg:squarederror, random_state=42, reg_alpha=0.015599452033620266, reg_lambda=1.5227525095137953, subsample=0.9464704583099741; total time=   0.3s
[CV] END colsample_bytree=0.5621780831931538, gamma=0.4753571532049581, learning_rate=0.14907884894416698, max_depth=7, min_child_weight=7, n_estimators=221, n_jobs=-1, objective=reg:squarederror, random_state=42, reg_alpha=0.015599452033620266, reg_lambda=1.5227525095137953, subsample=0.9464704583099741; total time=   0.3s
[CV] END colsample_bytree=0.5621780831931538, gamma=0.4753571532049581, learning_rate=0.14907884894416698, max_depth=7, min_child_weight=7, n_estimators=221, n_jobs=-1, objective=reg:squarederror, random_state=42, reg_alpha=0.015599452033620266, reg_lambda=1.5227525095137953, subsampl

### The Final Root Mean Squared Error of the Tuned XGBoost Regressor Model = 29025.80

### This is an improvement from the non-Tuned RMSE of 29198.292

___
## **Housing Dataset Tuned XGBoost Regressor Model Predictions using the Encoded Test Dataset** 
___

In [40]:
#store encoded dataset in variable for readability
testpath = ("/Users/sa14/Desktop/HousePricesKaggle/data/test_encoded_dataset.csv")

#load as a dataframe
test_encode = pd.read_csv(testpath)

#Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

test_encode.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_None,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_NA,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_NA,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NA,BsmtQual_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NA,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NA,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NA,BsmtFinType2_Rec,BsmtFinType2_Unf,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_NA,KitchenQual_TA,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NA,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,468.0,270.0,882.0,896,0,896,0.0,1,0,2,5,0,1961.0,1.0,730.0,140,0,6,2010,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,81.0,14267,6,6,1958,1958,923.0,406.0,1329.0,1329,0,1329,0.0,1,1,3,6,0,1958.0,1.0,312.0,393,36,6,2010,0.0,0.0,0.0,0.0,1.0,0.0,1.0

In [41]:
#Verify rows and columns
test_encode.shape

(1459, 190)

In [47]:
# ROBUST Scale encoded TRAINING dataset
from sklearn.preprocessing import RobustScaler

#ROBUST Scale Test Encoded Data
X2_new = scaler.fit_transform(test_encode)

# Predict SalePrices
sale_predictions = final_trained_model.predict(X2_new)

In [51]:
#Print Home IDs and Predicted SalePrice of the first 10 homes from the Encoded Test Dataset
sample_submission_df = pd.read_csv("/Users/sa14/Desktop/HousePricesKaggle/data/sample_submission.csv")
sample_submission_df['SalePrice'] = (final_trained_model.predict(X2_new))
sample_submission_df.to_csv('saleprice_predictions_thetechleila.csv', index=False)
sample_submission_df.head(10)

,Id,SalePrice
0,1461,132738.140625
1,1462,173444.984375
2,1463,193664.109375
3,1464,213597.890625
4,1465,184206.296875
5,1466,184748.625000
6,1467,189214.187500
7,1468,191423.375000
8,1469,173896.250000
9,1470,126868.781250


### **View the full Home SalePrice Predictions in [this](https://github.com/thetechleila/Housing-Prices-Predictor/blob/main/data_101/saleprice_predictions_thetechleila.csv) csv file.**